# Midterm

The midterm project will consist of a comparison between several CNN architectures for organ segmentation. The goal is both to create a high-performing algorithm for the target task, as well as to analyze performance across several different architecture permutations. In total, three different network designs will be tested. As each model is built and trained, ensure to serialize the final model `*.hdf5` file before moving to the next iteration.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model files for all three models
* final compiled `*.csv` file with performance statistics across the different architectures
* final 1-page write-up with methods and results of experiments

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 89 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about the KiTS Challenge can be found here: https://kits21.kits-challenge.org/. In this exercise, we will use this dataset to derive a model for kidney segmentation. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [3]:
# --- Download dataset
datasets.download(name='ct/kits')

[ 2022-05-04 01:03:33 ] [====================] 100.000% : Extracting archive (0000818 / 0000818) 

{'code': '/data/raw/ct_kits', 'data': '/data/raw/ct_kits'}

Since the algorithms below may require slightly different model inputs, the required generators and inputs will be defined dyanically in the code blocks later in this notebook.

# Training

A total of three different network architectures will be tested. The goal is to compare the incremental benefit of several design choices. After building and training each model to convergence, do not forget to save each model as a separate `*.hdf5` file.

## 1. 2D U-Net

In this algorithm a standard 2D U-Net architecture will be used to perform organ segmentation. The algorithm input will include an `96 x 96` resolution 2D slice from an abdominal CT exam. Key customizations to the standard U-Net architecture that should be implemented (as in the week 5 and week 6 tutorials) include:

* same padding (vs. valid padding)
* strided convolutions (vs. max-pooling)

### 2D Lambda and Functions

```
# This is formatted as code
```



In [4]:
# ---- kwargs dic, lambda, se func, dsc func

kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'
}

conv = lambda x, filters, strides : layers.Conv3D(filters = filters, strides = strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides = 1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides = (1, 2, 2))))

tran = lambda x, filters, strides : layers.Conv3DTranspose(filters = filters, strides = strides, **kwargs)(x)
tran2 = lambda filters, x: relu(norm(tran(x, filters, strides = (1,2 ,2))))

concat = lambda a, b : layers.Concatenate()([a, b])

def se(layer):
  
  sqz = layers.AveragePooling3D((1, layer.shape[2], layer.shape[3]))(layer)
  cha = int(layer.shape[-1]/4)
  exc = layers.Conv3D(filters = cha, kernel_size = 1, activation = 'relu')(sqz)
  sca = layers.Conv3D(filters = layer.shape[-1], kernel_size = 1, activation = 'sigmoid')(exc)

  return layer * sca

def calc_dsc(y_true, y_pred, c=1):

  true = y_true[..., 0] == c
  pred = tf.math.argmax(y_pred, axis =-1) == c

  A = tf.math.count_nonzero(true & pred) * 2
  B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)

  return tf.math.divide_no_nan(
      tf.cast(A, tf.float32),
      tf.cast(B, tf.float32)
  )

### Create generators and inputs

In [5]:
# --- Input ==> 1 x 96 x 96 x 1
configs = {'batch': {'size': 16}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d-bin', configs=configs, custom_layers=True)

### Define model

In [6]:
# --- Create backbone model

x = Input(shape=(None, 96, 96, 1), dtype='float32')


l1 = conv1(8, x)

l2 = conv1(16, conv2(16, l1))

l3 = conv1(32, conv2(32,l2))

l4 = conv1(48, conv2(48, l3))

l5 = conv1(64, conv2(64, l4))

l6 = tran2(48, l5)

l7 = tran2(32, conv1(48, concat(l4, l6))) 

l8 = tran2(16, conv1(32, concat(l3, l7)))

l9 = tran2(8, conv1(16, concat(l2, l8)))

l10 = conv1(8, l9)

logits = layers.Conv3D(filters = 2, **kwargs)(l10)

backbone = Model(inputs=x, outputs=logits)

In [7]:
# --- Create training model

inputs = {
    'dat': Input(shape=(None, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(None, 96, 96, 1), name='lbl')}

logits = backbone(inputs['dat'])

sce = losses.SparseCategoricalCrossentropy(from_logits=True)
loss = sce(y_true=inputs['lbl'], y_pred=logits)

dsc = calc_dsc(y_true=inputs['lbl'], y_pred=logits)

training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'dsc': dsc})

training.add_loss(loss)
training.add_metric(dsc, name = 'dsc')

optimizer = optimizers.Adam(learning_rate=2e-4)


### Compile and train model

In [8]:
# --- Compile model
training.compile(optimizer=optimizer)

client.load_data_in_memory()

# --- Train the model
training.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=10,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=5)

[ 2022-05-04 01:04:00 ] [====================] 100.000% : Iterating | 000402    Epoch 1/10
100/100 [==============================] - 19s 43ms/step - loss: 0.3948 - dsc: 0.1170
Epoch 2/10
100/100 [==============================] - 4s 39ms/step - loss: 0.1849 - dsc: 0.4017
Epoch 3/10
100/100 [==============================] - 4s 37ms/step - loss: 0.1239 - dsc: 0.6765
Epoch 4/10
100/100 [==============================] - 4s 39ms/step - loss: 0.0782 - dsc: 0.9012
Epoch 5/10
100/100 [==============================] - 7s 66ms/step - loss: 0.0550 - dsc: 0.9193 - val_loss: 0.0655 - val_dsc: 0.8874
Epoch 6/10
100/100 [==============================] - 3s 26ms/step - loss: 0.0436 - dsc: 0.9346
Epoch 7/10
100/100 [==============================] - 3s 29ms/step - loss: 0.0391 - dsc: 0.9390
Epoch 8/10
100/100 [==============================] - 4s 36ms/step - loss: 0.0349 - dsc: 0.9426
Epoch 9/10
100/100 [==============================] - 4s 38ms/step - loss: 0.0319 - dsc: 0.9451
Epoch 10/10
100/10

In [9]:
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc_2d_valid = []
dsc_2d_train = []

for x, _ in test_valid:
    
    # --- Predict
    outputs = training.predict(x)

    # --- Argmax
    dsc_2d_valid.append(outputs['dsc'])

dsc_2d_valid = np.array(dsc_2d_valid)

for x, _ in test_train:
    
    # --- Predict
    outputs = training.predict(x)

    # --- Argmax
    dsc_2d_train.append(outputs['dsc'])

dsc_2d_train = np.array(dsc_2d_train)

[ 2022-05-04 01:05:53 ] [====================] 100.000% : Iterating | 000321    

In [10]:
twod_df_train = pd.DataFrame(index=np.arange(dsc_2d_train.size))
twod_df_train['Dice score'] = dsc_2d_train

print(twod_df_train['Dice score'].mean())

twod_df_valid = pd.DataFrame(index=np.arange(dsc_2d_valid.size))
twod_df_valid['Dice score'] = dsc_2d_valid

print(twod_df_valid['Dice score'].mean())

0.9536712188215642
0.930788650188917


In [11]:
twod_df_train.to_csv('./dice_results_2D_train.csv')
twod_df_valid.to_csv('./dice_results_2D_valid.csv')


In [12]:
backbone.save('./wjhan_2Dmodel.hdf5')

## 2. 3D U-Net

In this algorithm, the original 2D model is modified to a full 3D model. The algorithm input will include an `96 x 96 x 96` resolution 3D volume from an abdominal CT exam. To ensure a fair comparison, recommend using similar network design and training hyperparameters as in the first 2D only model above. 

### Create generators and inputs

In [13]:
# --- Input ==> 96 x 96 x 96 x 1
configs = {'batch': {'size': 16}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d-bin', configs=configs, custom_layers=True)

### 3D Lambda

In [14]:
# ---- kwargs dic, lambda, se func, dsc func

kwargs = {
    'kernel_size': (3, 3, 3),
    'padding': 'same',
    'kernel_initializer': 'he_normal'
}

conv = lambda x, filters, strides : layers.Conv3D(filters = filters, strides = strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides = 1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides = (2, 2, 2))))

tran = lambda x, filters, strides : layers.Conv3DTranspose(filters = filters, strides = strides, **kwargs)(x)
tran2 = lambda filters, x: relu(norm(tran(x, filters, strides = (2,2 ,2))))

concat = lambda a, b : layers.Concatenate()([a, b])

### Define model

In [15]:
# --- Create backbone model

x = Input(shape=(96, 96, 96, 1), dtype='float32')

l1 = conv1(8, x)

l2 = conv1(16, conv2(16, l1))

l3 = conv1(32, conv2(32, l2))

l4 = conv1(48, conv2(48, l3))

l5 = conv1(64, conv2(64, l4))


l6  = tran2(48, l5)

l7  = tran2(32, conv1(48, concat(l4, l6)))

l8  = tran2(16, conv1(32, concat(l3, l7)))

l9  = conv1(8, tran2(8,  conv1(16, concat(l2, l8))))

logits = layers.Conv3D(filters=2, **kwargs)(l9)


logits = {
    
    'c0': layers.Conv3D(filters=2, **kwargs)(l9),
    'c1': layers.Conv3D(filters=2, **kwargs)(l8),
}
backbone = Model(inputs = x, outputs = logits)

In [16]:
# --- Create training model
inputs = {
    'dat': Input(shape=(96, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(96, 96, 96, 1), name='lbl')}
logits = backbone(inputs['dat'])


loss = {}
true = inputs['lbl']

for c in sorted(logits.keys()):
    
    if c != 'c0':
        true = layers.MaxPooling3D(pool_size=(2, 2, 2))(true)
    
    loss[c] = losses.SparseCategoricalCrossentropy(from_logits=True, name='sce-' + c)(
        y_true=true,
        y_pred=logits[c])


dsc = calc_dsc(y_true=inputs['lbl'], y_pred=logits['c0'])

training = Model(inputs=inputs, outputs={**logits, **loss, **{'dsc': dsc}})

for l in loss.values():
    training.add_loss(l)

training.add_metric(dsc, name='dsc')

optimizer = optimizers.Adam(learning_rate=2e-4)

### Compile and train model

In [17]:
# --- Compile model
training.compile(optimizer = optimizer)

client.load_data_in_memory()
# --- Train the model
training.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=10,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=5)

[ 2022-05-04 01:08:29 ] [====================] 100.000% : Iterating | 000402    Epoch 1/10
100/100 [==============================] - 115s 1s/step - loss: 0.5717 - dsc: 0.5263
Epoch 2/10
100/100 [==============================] - 108s 1s/step - loss: 0.1110 - dsc: 0.9350
Epoch 3/10
100/100 [==============================] - 108s 1s/step - loss: 0.0621 - dsc: 0.9564
Epoch 4/10
100/100 [==============================] - 107s 1s/step - loss: 0.0445 - dsc: 0.9644
Epoch 5/10
100/100 [==============================] - 132s 1s/step - loss: 0.0353 - dsc: 0.9689 - val_loss: 0.0736 - val_dsc: 0.9175
Epoch 6/10
100/100 [==============================] - 107s 1s/step - loss: 0.0292 - dsc: 0.9729
Epoch 7/10
100/100 [==============================] - 108s 1s/step - loss: 0.0257 - dsc: 0.9743
Epoch 8/10
100/100 [==============================] - 109s 1s/step - loss: 0.0233 - dsc: 0.9756
Epoch 9/10
100/100 [==============================] - 108s 1s/step - loss: 0.0210 - dsc: 0.9772
Epoch 10/10
100/100

In [18]:
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc_3d_valid = []
dsc_3d_train = []

for x, _ in test_valid:
    
    # --- Predict
    outputs = training.predict(x)

    # --- Argmax
    dsc_3d_valid.append(outputs['dsc'])

dsc_3d_valid = np.array(dsc_3d_valid)


for x, _ in test_train:
    
    # --- Predict
    outputs = training.predict(x)

    # --- Argmax
    dsc_3d_train.append(outputs['dsc'])

dsc_3d_train = np.array(dsc_3d_train)

[ 2022-05-04 01:28:27 ] [====================] 100.000% : Iterating | 000321    

In [19]:
threed_df_train = pd.DataFrame(index=np.arange(dsc_3d_train.size))
threed_df_train['Dice score'] = dsc_3d_train

print(threed_df_train['Dice score'].mean())

threed_df_valid = pd.DataFrame(index=np.arange(dsc_3d_valid.size))
threed_df_valid['Dice score'] = dsc_3d_valid

print(threed_df_valid['Dice score'].mean())

0.9639075910933664
0.9334891850565686


In [20]:
threed_df_train.to_csv('./dice_results_3D_train.csv')
threed_df_valid.to_csv('./dice_results_3D_valid.csv')
backbone.save('./wjhan_3Dmodel.hdf5')

## 3. Custom architecture

Finally, using any of the customizations described in class, find a top-performing model that may potentially yield some incremental benefit over the two baseline models above. Modifications that may be used include (but are not limited to):

* deep supervision
* residual connections
* added convolutions between contracting and expanding layers 
* modifications to the convolutional blocks including ResNet, Inception, SE-Net

### Create generators and inputs

In [21]:
# --- Choose input (may copy the generator code from above)
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d-bin', custom_layers=True)

### Custom Architecture Lambda and Functions

In [22]:
# ---- kwargs dic, lambda, se func, dsc func

kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'
}

conv = lambda x, filters, strides : layers.Conv3D(filters = filters, strides = strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides = 1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides = (1, 2, 2))))

tran = lambda x, filters, strides : layers.Conv3DTranspose(filters = filters, strides = strides, **kwargs)(x)
tran2 = lambda filters, x: relu(norm(tran(x, filters, strides = (1,2 ,2))))

concat = lambda a, b : layers.Concatenate()([a, b])

def se(layer):
  
  sqz = layers.AveragePooling3D((1, layer.shape[2], layer.shape[3]))(layer)
  cha = int(layer.shape[-1]/4)
  exc = layers.Conv3D(filters = cha, kernel_size = 1, activation = 'relu')(sqz)
  sca = layers.Conv3D(filters = layer.shape[-1], kernel_size = 1, activation = 'sigmoid')(exc)

  return layer * sca

def calc_dsc(y_true, y_pred, c=1):

  true = y_true[..., 0] == c
  pred = tf.math.argmax(y_pred, axis =-1) == c

  A = tf.math.count_nonzero(true & pred) * 2
  B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)

  return tf.math.divide_no_nan(
      tf.cast(A, tf.float32),
      tf.cast(B, tf.float32)
  )

### Define model

In [23]:
# --- Create backbone model
x = Input(shape=(None, 96, 96, 1), dtype='float32')


l1 = conv1(8, x)
l1 = se(l1)

l2 = conv1(16, conv2(16, l1))
l2 = se(l2)

l3 = conv1(32, conv2(32,l2))
l3 = se(l3)

l4 = conv1(48, conv2(48, l3))
l4 = se(l4)

l5 = conv1(64, conv2(64, l4))
l5 = se(l5)

l6 = tran2(48, l5)

l7 = tran2(32, conv1(48, concat(l4, l6))) 

l8 = tran2(16, conv1(32, concat(l3, l7)))

l9 = tran2(8, conv1(16, concat(l2, l8)))

l10 = conv1(8, l9)




logits = layers.Conv3D(filters = 2, **kwargs)(l10)

# --- Create model
backbone = Model(inputs=x, outputs=logits)


In [24]:
# --- Create training model
inputs = {
    'dat': Input(shape= (None, 96, 96, 1), name = 'dat'),
    'lbl': Input(shape = (None, 96, 96, 1), name = 'lbl')}

logits = backbone(inputs['dat'])

sce = losses.SparseCategoricalCrossentropy(from_logits = True)
loss = sce(y_true=inputs['lbl'], y_pred = logits)

dsc = calc_dsc(y_true = inputs['lbl'], y_pred = logits)

training = Model(inputs=inputs, outputs={'logits': logits, 'loss' : loss, 'dsc' : dsc})

training.add_loss(loss)

training.add_metric(dsc, name = 'dsc')

optimizer = optimizers.Adam(learning_rate = 2e-4)


### Compile and train model

In [25]:
# --- Compile model
training.compile(optimizer = optimizer)
client.load_data_in_memory()
# --- Train the model
training.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=10,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=5)

[ 2022-05-04 01:28:37 ] [====================] 100.000% : Iterating | 000402    Epoch 1/10
100/100 [==============================] - 6s 28ms/step - loss: 0.5710 - dsc: 0.1988
Epoch 2/10
100/100 [==============================] - 3s 28ms/step - loss: 0.2360 - dsc: 0.7378
Epoch 3/10
100/100 [==============================] - 3s 28ms/step - loss: 0.1201 - dsc: 0.8656
Epoch 4/10
100/100 [==============================] - 3s 28ms/step - loss: 0.0813 - dsc: 0.8933
Epoch 5/10
100/100 [==============================] - 5s 53ms/step - loss: 0.0615 - dsc: 0.9145 - val_loss: 0.0889 - val_dsc: 0.8161
Epoch 6/10
100/100 [==============================] - 3s 28ms/step - loss: 0.0504 - dsc: 0.9248
Epoch 7/10
100/100 [==============================] - 3s 29ms/step - loss: 0.0424 - dsc: 0.9338
Epoch 8/10
100/100 [==============================] - 3s 28ms/step - loss: 0.0395 - dsc: 0.9359
Epoch 9/10
100/100 [==============================] - 3s 28ms/step - loss: 0.0356 - dsc: 0.9414
Epoch 10/10
100/100

In [26]:
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc_custom_valid = []
dsc_custom_train = []

for x, _ in test_valid:
    
    # --- Predict
    outputs = training.predict(x)

    # --- Argmax
    dsc_custom_valid.append(outputs['dsc'])

dsc_custom_valid = np.array(dsc_custom_valid)


for x, _ in test_train:
    
    # --- Predict
    outputs = training.predict(x)

    # --- Argmax
    dsc_custom_train.append(outputs['dsc'])

dsc_custom_train = np.array(dsc_custom_train)

[ 2022-05-04 01:29:53 ] [====================] 100.000% : Iterating | 000321    

In [27]:
custom_df_train = pd.DataFrame(index=np.arange(dsc_custom_train.size))
custom_df_train['Dice score'] = dsc_custom_train

print(custom_df_train['Dice score'].mean())

custom_df_valid = pd.DataFrame(index=np.arange(dsc_custom_valid.size))
custom_df_valid['Dice score'] = dsc_custom_valid

print(custom_df_valid['Dice score'].mean())

0.9498334205781931
0.9234169717924094


In [28]:
custom_df_train.to_csv('./dice_results_custom_train.csv')
custom_df_valid.to_csv('./dice_results_custom_valid.csv')
backbone.save('./wjhan_custommodel.hdf5')

# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* Dice score, mean
* Dice score, median
* Dice score, 25th percentile
* Dice score, 75th percentile

As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the Dice score values on the entire 3D volume (not slice-by-slice).

### Performance

The following minimum **validation cohort** performance metrics must be met for full credit:

1. **2D U-Net**: mean Dice score > 0.80
2. **3D U-Net**: mean Dice score > 0.82
3. **Custom architecture**: mean Dice score > 0.84

**Bonus**: any final model with a mean **validation cohort** Disce score > 0.96 will recieve a +5 point (+15%) extra credit towards the midterm assignment.

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the three different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          TRAINING                                VALIDATION
          mean | median | 25th-tile | 75th-tile | mean | median | 25th-tile | 75th-tile
model 1
model 2
model 3
```

As above, statistics for both training and validation should be provided.

In [31]:
!ls

dice_results_2D_train.csv      dice_results_custom_valid.csv
dice_results_2D_valid.csv      sample_data
dice_results_3D_train.csv      wjhan_2Dmodel.hdf5
dice_results_3D_valid.csv      wjhan_3Dmodel.hdf5
dice_results_custom_train.csv  wjhan_custommodel.hdf5


In [52]:
customdf_train = pd.read_csv('dice_results_custom_train.csv')
customdf_valid = pd.read_csv('dice_results_custom_valid.csv')

threeddf_train = pd.read_csv('dice_results_3D_train.csv')
threeddf_valid = pd.read_csv('dice_results_3D_valid.csv')

twoddf_train = pd.read_csv('dice_results_2D_train.csv')
twoddf_valid = pd.read_csv('dice_results_2D_valid.csv')

print('train mean')

print(customdf_train['Dice score'].mean())
print(threeddf_train['Dice score'].mean())
print(twoddf_train['Dice score'].mean())


print('train median')

print(customdf_train['Dice score'].median())
print(threeddf_train['Dice score'].median())
print(twoddf_train['Dice score'].median())


print('train 25th')

print(customdf_train['Dice score'].quantile(.25))
print(threeddf_train['Dice score'].quantile(.25))
print(twoddf_train['Dice score'].quantile(.25))


print('train 75th')

print(customdf_train['Dice score'].quantile(.75))
print(threeddf_train['Dice score'].quantile(.75))
print(twoddf_train['Dice score'].quantile(.75))


print('valid mean')

print(customdf_valid['Dice score'].mean())
print(threeddf_valid['Dice score'].mean())
print(twoddf_valid['Dice score'].mean())


print('valid median')

print(customdf_valid['Dice score'].median())
print(threeddf_valid['Dice score'].median())
print(twoddf_valid['Dice score'].median())


print('valid 25th')
print(customdf_valid['Dice score'].quantile(.25))
print(threeddf_valid['Dice score'].quantile(.25))
print(twoddf_valid['Dice score'].quantile(.25))


print('valid 75th')
print(customdf_valid['Dice score'].quantile(.75))
print(threeddf_valid['Dice score'].quantile(.75))
print(twoddf_valid['Dice score'].quantile(.75))

train mean
0.9498334205781931
0.9639075910933664
0.9536712188215642
train median
0.9609836935997008
0.9699203372001648
0.9638468623161316
train 25th
0.9460278153419496
0.9612881541252136
0.9500337839126588
train 75th
0.9707959294319152
0.9750203490257264
0.9727936387062072
valid mean
0.9234169717924094
0.9334891850565686
0.930788650188917
valid median
0.956113874912262
0.956473171710968
0.9561198949813844
valid 25th
0.9116793870925904
0.9358364343643188
0.9258504509925842
valid 75th
0.9668260216712952
0.96698796749115
0.9693339467048644


In [57]:
import pandas as pd

# --- Create *.csv
df_dict = {
    'models'       : ['Custom', '3D', '2D'],
    'mean_train'   : [0.9498334205781931, 0.9639075910933664, 0.9536712188215642], 
    'median_train' : [0.9609836935997008, 0.9699203372001648, 0.9638468623161316], 
    '25th_train'   : [0.9460278153419496, 0.9612881541252136, 0.9500337839126588],
    '75th_train'   : [0.9707959294319152, 0.9750203490257264, 0.9727936387062072],
    'mean_valid'   : [0.9234169717924094, 0.9334891850565686, 0.930788650188917],
    'median_valid' : [0.956113874912262, 0.956473171710968, 0.9561198949813844], 
    '25th_valid'   : [0.9116793870925904, 0.9358364343643188, 0.9258504509925842],
    '75th_valid'   : [0.9668260216712952, 0.96698796749115, 0.9693339467048644]
}

df = pd.DataFrame(df_dict)



# --- Serialize *.csv
df.to_csv('./wjhan_results.csv')

# Summary

In addition to algorithm training as above, a 1-2 page write-up is required for this project. The goal is to *briefly* summarize algorithm design and key results. The write-up should be divided into three sections: methods; results; discussion. More detailed information and tips can be found here: https://github.com/peterchang77/dl_tutor/blob/master/cs190/spring_2021/notebooks/midterm/checklist.md.

### Methods

In this section, include details such as:

* **Data**: How much data was used. How many cases were utilized for training and validation?
* **Network design**: What are the different network architectures? How many layers and parameters? Were 2D or 3D operations used? Recall that the `model.summary(...)` can be used to provide key summary statistics for this purpose. If desired, feel free to include a model figure or diagram.
* **Implementation**: How was training implemented. What are the key hyperparameters (e.g. learning rate, batch size, optimizer, etc)? How many training iterations were required for convergence? Did these hyperparameters change during the course of training?
* **Statistics**: What statistics do you plan to use to evaluate model accuracy? 

### Results

In this section, briefly summarize experimental results (a few sentences), and include the result table(s) as derived above.

### Discussion

Were the results expected or unexpected? What accounts for the differences in performance between the algorithms? How did you choose the network architecture implemented in your final model? Feel free to elaborate on any additional observations noted during the course of this expierment.

# Submission


### Canvas

Once you have completed the midterm assignment, download the necessary files from Google Colab and your Google Drive. As in prior assigments, be sure to prepare:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv` (compiled for all three parts)
* final (trained) model: `[UCInetID]_model.hdf5` (three separate files for all three parts)

In addition, submit the summary write-up as in any common document format (`.docx`, `.tex`, `.pdf`, etc):

* final summary write-up: `[UCInetID]_summary.[docx|tex|pdf]`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadsheet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.